In [ ]:
import os
import json
import time
import pytz
from datetime import datetime
import requests
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import LineString
import networkx as nx
import hashlib
import json
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

import scripts.TrafficModelWithJunctionRespect as tmjr
import scripts.EvolutionaryOptimizationParallel as evo

import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import re
import os
import pickle
from natsort import natsorted

from scipy.interpolate import interp1d

In [ ]:
PATH = "ISTANBUL"
timeSection = "morning"
t = "0"

DAY = 0
DAYLIMIT = 1

In [ ]:
def parse_timestamp_from_filename(filename):
    """
    Given a filename like 'traffic_data_20250214_230741.json' or 'traffic__20250214_230741.json',
    extract the date/time part and return an ISO-like string:
    '2025-02-14T23:07:41Z'
    """
    match = re.search(r"(?:traffic_data_|traffic__|traffic_)(\d{8})_(\d{6})\.json", filename)
    if not match:
        raise ValueError(f"Filename format not recognized: {filename}")

    date_str, time_str = match.group(1), match.group(2)
    dt = datetime.strptime(date_str + time_str, "%Y%m%d%H%M%S")
    return dt.isoformat() + 'Z'

# --- Load data files ---
with open(os.path.join(PATH, "city_file_lists.pkl"), "rb") as f:
    cityFileLists = pickle.load(f)

selected_days = list(cityFileLists[PATH]['days'].keys())[1:1 + DAYLIMIT]

timezone_map = {
    "ISTANBUL": pytz.timezone("Europe/Istanbul"),
    "LONDON": pytz.timezone("Europe/London"),
    "NY": pytz.timezone("America/New_York"),
    "TOKYO": pytz.timezone("Asia/Tokyo"),
    "TOKYO2": pytz.timezone("Asia/Tokyo"),
    "LONDON2": pytz.timezone("Europe/London"),
    "BERLIN": pytz.timezone("Europe/Berlin"),
}

# --- Load WTs ---
files_per_time = {"morning": []}
for section in files_per_time:
    section_path = os.path.join(PATH, section)
    for fname in sorted(os.listdir(section_path)):
        with open(os.path.join(section_path, fname), "rb") as f:
            data = pickle.load(f)
            files_per_time[section].append(data)

files_per_time = files_per_time[timeSection]
WTsForDifTimes = []

for file in files_per_time[:DAYLIMIT]:
    G, C, L, W, dataAvailableIndices = file
    WTsForDifTimes.append(W)

# --- Interpolate ---
interpolated_WTs = []
timestamps_fine_all = []

for day_index, WT in enumerate(WTsForDifTimes):
    T, E = WT.shape

    # 1. Get timestamps from filenames
    date_key = selected_days[day_index]
    files = cityFileLists[PATH]['days'][date_key][timeSection]
    files.sort(key=parse_timestamp_from_filename)

    base_times = []
    istanbul_tz = pytz.timezone("Europe/Istanbul")
    target_tz = timezone_map[PATH]

    for fname in files:
        timestamp_str = parse_timestamp_from_filename(fname)

        # Parse as naive datetime (because it's Istanbul-local time)
        dt_naive = datetime.strptime(timestamp_str, "%Y-%m-%dT%H:%M:%SZ")

        # Localize as Istanbul time
        dt_istanbul = istanbul_tz.localize(dt_naive)

        # Convert to local time of the city the data represents
        dt_local = dt_istanbul.astimezone(target_tz)

        base_times.append(dt_local)

    # 2. Interpolate WTs
    time_original = np.arange(T)
    time_fine = np.linspace(0, T - 1, (T - 1) * 5 + 1)
    interpolated = np.zeros((len(time_fine), E))

    for e in range(E):
        f = interp1d(time_original, WT[:, e], kind='linear', fill_value="extrapolate")
        interpolated[:, e] = f(time_fine)

    # 3. Interpolate timestamps
    interpolated_times = []
    for t in time_fine:
        low_idx = int(np.floor(t))
        high_idx = min(int(np.ceil(t)), len(base_times) - 1)
        frac = t - low_idx
        t_interp = base_times[low_idx] + (base_times[high_idx] - base_times[low_idx]) * frac
        interpolated_times.append(t_interp.strftime('%Y-%m-%d %H:%M'))

    # 4. Append results
    interpolated_WTs.append(interpolated)
    timestamps_fine_all.append(interpolated_times)

# --- Save output ---
with open(os.path.join(PATH, f"interpolated_WTs_{timeSection}.pkl"), "wb") as f:
    pickle.dump(interpolated_WTs, f)

with open(os.path.join(PATH, f"interpolated_timestamps_{timeSection}.pkl"), "wb") as f:
    pickle.dump(timestamps_fine_all, f)

with open(os.path.join(PATH, "G.pkl"), "wb") as f:
    pickle.dump(G, f)

with open(os.path.join(PATH, "C.pkl"), "wb") as f:
    pickle.dump(C, f)

with open(os.path.join(PATH, "dataAvailableIndices.pkl"), "wb") as f:
    pickle.dump(dataAvailableIndices, f)

In [ ]:
morning_files_collection = [] 

for i in os.listdir(os.path.join(PATH, "morning")):
    with open(os.path.join(os.path.join(PATH, "morning"), i), "rb") as f:
        morning_files = pickle.load(f)
    morning_files_collection.append(morning_files)

night_files_collection = []

# for i in os.listdir(os.path.join(PATH, "night")):
#     with open(os.path.join((os.path.join(PATH, "night")), i), "rb") as f:
#         night_files = pickle.load(f)
#     night_files_collection.append(night_files)

files_per_time = {"morning" : morning_files_collection, "night" : night_files_collection}
files_per_time = files_per_time[timeSection]

WTsForDifTimes = []

for file in files_per_time[:DAYLIMIT]:
    G,C,L,W, dataAvailableIndices = file
    WTsForDifTimes.append(W)
    

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Create dummy WTsForDifTimes: list of 3 arrays, each (time, edge) = (20, 5)

# Interpolate to 1-minute resolution (from 5-minute sampling)
interpolated_WTs = []

for WT in WTsForDifTimes:
    T, E = WT.shape
    time_original = np.arange(T)
    time_fine = np.linspace(0, T - 1, (T - 1) * 5 + 1)  # 1-min intervals

    interpolated = np.zeros((len(time_fine), E))
    for e in range(E):
        f = interp1d(time_original, WT[:, e], kind='linear', fill_value="extrapolate")
        interpolated[:, e] = f(time_fine)

    interpolated_WTs.append(interpolated)

# Plot original and interpolated for the first array
fig, axs = plt.subplots(5, 1, figsize=(10, 10), sharex=True)

WT0 = WTsForDifTimes[0]
nonzero_edges = [e for e in range(WT0.shape[1]) if np.any(WT0[:, e] > 0)]
for i, edge in enumerate(nonzero_edges[:5]):
    axs[i].plot(np.arange(WT0.shape[0]) * 5, WT0[:, edge], 'o-', label='Original (5-min)')
    axs[i].plot(np.linspace(0, (WT0.shape[0] - 1) * 5, interpolated_WTs[0].shape[0]),
                interpolated_WTs[0][:, edge], '-', label='Interpolated (1-min)')
    axs[i].set_ylabel(f'Edge {edge}')
    axs[i].legend()


axs[-1].set_xlabel('Minutes')
plt.tight_layout()
plt.show()


In [ ]:
# save interpolated_WTs to a file
with open(os.path.join(PATH, f"interpolated_WTs_{timeSection}.pkl"), "wb") as f:
    pickle.dump(interpolated_WTs, f)

# save G 
with open(os.path.join(PATH, f"G.pkl"), "wb") as f:
    pickle.dump(G, f)

# save C
with open(os.path.join(PATH, f"C.pkl"), "wb") as f:
    pickle.dump(C, f)   

with open(os.path.join(PATH, f"dataAvailableIndices.pkl"), "wb") as f:
    pickle.dump(dataAvailableIndices, f)